In [9]:
import tensorflow as tf
import numpy as np
import os, argparse, time, random
from utils import str2bool, get_logger, get_entity
from data import read_corpus, read_dictionary, tag2label, random_embedding

from model import BiLSTM_CRF

In [2]:
## Session configuration
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # default: 0
config = tf.ConfigProto()

In [3]:
## paths setting
paths = {}
timestamp='1526890202'
output_path = os.path.join('.', 'data_path'+"_save", timestamp)
if not os.path.exists(output_path): os.makedirs(output_path)
summary_path = os.path.join(output_path, "summaries")
paths['summary_path'] = summary_path
if not os.path.exists(summary_path): os.makedirs(summary_path)
model_path = os.path.join(output_path, "checkpoints/")
if not os.path.exists(model_path): os.makedirs(model_path)
ckpt_prefix = os.path.join(model_path, "model")
paths['model_path'] = ckpt_prefix
result_path = os.path.join(output_path, "results")
paths['result_path'] = result_path
if not os.path.exists(result_path): os.makedirs(result_path)
log_path = os.path.join(result_path, "log.txt")
paths['log_path'] = log_path

In [4]:
## get char embeddings
word2id = read_dictionary(os.path.join('.', 'data_path', 'word2id.pkl'))

vocab_size: 3905


In [5]:
embeddings = np.load('pretrain_embedding.npy')

In [6]:
test_path = os.path.join('.', 'data_path', 'test_data')
test_data = read_corpus(test_path)
test_size = len(test_data)

In [7]:
## testing model
# elif args.mode == 'test':
#     ckpt_file = tf.train.latest_checkpoint(model_path)
#     print(ckpt_file)
#     paths['model_path'] = ckpt_file
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()
#     print("test data: {}".format(test_size))
#     model.test(test_data)
## testing model
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
model.build_graph()
print("test data: {}".format(test_size))
model.test(test_data)

.\data_path_save\1526890202\checkpoints/model-31680


C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
=========== testing ===========


test data: 4631
INFO:tensorflow:Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


In [8]:
## demo
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
model.build_graph()
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    print('============= demo =============')
    saver.restore(sess, ckpt_file)
    while(1):
        print('Please input your sentence:')
        demo_sent = input()
        if demo_sent == '' or demo_sent.isspace():
            print('See you next time!')
            break
        else:
            demo_sent = list(demo_sent.strip())
            demo_data = [(demo_sent, ['O'] * len(demo_sent))]
            tag = model.demo_one(sess, demo_data)
            PER, LOC, ORG = get_entity(tag, demo_sent)
            print('PER: {}\nLOC: {}\nORG: {}'.format(PER, LOC, ORG))

.\data_path_save\1526890202\checkpoints/model-31680


C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


============= demo =============
INFO:tensorflow:Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


Please input your sentence:
詹姆斯已经命中了6次投篮，生涯季后赛命中数超越天勾贾巴尔，升至历史第一
PER: ['詹姆斯', '贾巴尔']
LOC: []
ORG: []
Please input your sentence:
大家好，我叫刘知远，来自清华大学计算机系。非常感谢前面李涓子老师、蔡老师讲了一些认知和知识图谱相关的知识，这样我就可以接着他们讲我今天的题目。我的题目是：表示学习跟大家耳熟能详的深度学习有密切的联系。
PER: ['刘知远', '李涓子', '蔡']
LOC: []
ORG: ['清华大学']
Please input your sentence:
大家好，我叫刘知远，我的籍贯是山西省晋中市祁县，来自清华大学计算机系。非常感谢前面李涓子老师、蔡老师讲了一些认知和知识图谱相关的知识，这样我就可以接着他们讲我今天的题目。我的题目是：表示学习跟大家耳熟能详的深度学习有密切的联系。
PER: ['刘知远', '李涓子', '蔡']
LOC: ['山西省', '晋中市', '祁县']
ORG: ['清华大学']
Please input your sentence:
大家好，我叫刘知远，我的籍贯是山西省晋中市祁县的西六支乡里面的王村，来自清华大学计算机系。非常感谢前面李涓子老师、蔡刚刚老师讲了一些认知和知识图谱相关的知识，这样我就可以接着他们讲我今天的题目。我的题目是：表示学习跟大家耳熟能详的深度学习有密切的联系。
PER: ['刘知远', '李涓子', '蔡刚刚']
LOC: ['山西省', '晋中市', '祁县', '王村']
ORG: ['清华大学计算机系']
Please input your sentence:
大家好，我叫刘知远，我的籍贯是山西省晋中市祁县的西六支乡的王村，来自清华大学计算机系。我的儿子是祁县公安局局长。非常感谢前面李涓子老师、蔡刚刚老师讲了一些认知和知识图谱相关的知识，这样我就可以接着他们讲我今天的题目。我的题目是：表示学习跟大家耳熟能详的深度学习有密切的联系。
PER: ['刘知远', '王村', '李涓子', '蔡刚刚']
LOC: ['山西省', '晋中市', '祁县', '西六支乡']
ORG: ['清华大学', '祁县公安局']
Please input your sentence:


In [10]:
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
model.build_graph()
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    print('============= demo =============')
    saver.restore(sess, ckpt_file)
    while(1):
        print('Please input your sentence:')
        demo_sent = input()
        if demo_sent == '' or demo_sent.isspace():
            print('See you next time!')
            break
        else:
            demo_sent = list(demo_sent.strip())
            demo_data = [(demo_sent, ['O'] * len(demo_sent))]
            tag = model.demo_one(sess, demo_data)
            PER, LOC, ORG = get_entity(tag, demo_sent)
            print('PER: {}\nLOC: {}\nORG: {}'.format(PER, LOC, ORG))

.\data_path_save\1526890202\checkpoints/model-31680


C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


============= demo =============
INFO:tensorflow:Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


Please input your sentence:
大家好，我叫刘环环，我的籍贯是山西省晋中市祁县西六支乡王村，来自清华大学计算机系。我的儿子是祁县公安局局长。我老婆在祁县人民医院工作，我孙子在王村小学教书，非常感谢前面李涓子老师、蔡刚刚老师讲了一些认知和知识图谱相关的知识，这样我就可以接着他们讲我今天的题目。我的题目是：表示学习跟大家耳熟能详的深度学习有密切的联系。
PER: ['刘环环', '李涓子', '蔡刚刚']
LOC: ['山西省', '晋中市', '祁县', '西六支乡', '王村']
ORG: ['清华大学', '祁县公安局', '祁县人民医院']
Please input your sentence:
我孙子在王村小学教书
PER: []
LOC: []
ORG: []
Please input your sentence:
我孙子在西电教书
PER: []
LOC: []
ORG: []
Please input your sentence:
我孙子在西安电子科技大学教书
PER: []
LOC: []
ORG: ['西安电子科技大学']
Please input your sentence:

See you next time!
